In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import nltk  
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
import math as Math
from utils import *

In [ ]:
#Set the maximum number of mentees per mentor
max_mentees_per_mentor = 10 

In [ ]:



# Download the set of stop words  
nltk.download('stopwords')  
nltk.download('punkt')  
  
# Load stop words  
stop_words = set(stopwords.words('english'))  
stop_words.remove('not')



In [ ]:

# Function to remove stop words  
def remove_stop_words(text):  
    # Tokenize the text  
    words = word_tokenize(text)  
    # Remove stop words and return the cleaned text  
    return ' '.join([word for word in words if word.lower() not in stop_words])  

Load the model for embedding

In [ ]:

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)


In [ ]:

#function to embed the input
def embed(input):
  return model(input)


def score_similarity_between_mentor_mentee_goals(mentee_goal, mentor_goal):
    mentee_interests_cleaned = remove_stop_words(mentee_goal)
    mentor_interests_cleaned = remove_stop_words(mentor_goal)
    # embed the interests
    embeddings = model([mentee_goal, mentor_goal])
    # compute similarity scores of two embeddings
    cosine_similarity_orig = np.inner(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    #compute embeddings of the cleaned interests
    embeddings = model([mentor_interests_cleaned, mentee_interests_cleaned])
     # compute similarity scores of two embeddings
    cosine_similarity_cleaned = np.inner(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    #print(cosine_similarity_orig, cosine_similarity_cleaned)
    #Get  the maximum similarity score


    return max(cosine_similarity_orig, cosine_similarity_cleaned).round(4)

Read and clean the mentee submission

In [ ]:
#Read the mentee dataset
mentee_df = pd.read_excel('../data/input/mentee.xlsx', engine='openpyxl')


In [ ]:
mentee_df['mentee_timezone'].head()

In [ ]:
#apply the function to the timezone column and create a new column
mentee_df['mentee_timezone_num'] = mentee_df['mentee_timezone'].apply(assignnumericTimeZone)
# Split the 'interests' column into separate strings
mentee_df['mentee_split_interests'] = mentee_df['mentee_interests'].str.split(';')

Read and clean the mentor dataset

In [ ]:
#Read the mentor dataset
mentor_df = pd.read_excel('../data/input/mentor.xlsx', engine='openpyxl')


In [ ]:
#create finction assignCirclePreference thats returns value as 'Mid if string contains 'Mid' else if string contains 'Early' returns 'Early'
#otherwise returns 'Late'
def assignCirclePreference(circle):
    if 'Mid' in circle:
        return 'Mid'
    elif 'Early' in circle:
        return 'Early'
    else:
        return 'NA'


In [ ]:
#apply the function to the timezone column and create a new column
mentor_df['mentor_timezone_num'] = mentor_df['mentor_timezone'].apply(assignnumericTimeZone)

# Split the 'interests' column into separate strings
mentor_df['mentor_split_interests'] = mentor_df['mentor_interests'].str.split(';')

#change values on mentor circle preference column to 'Early' if value contains 'Early' and 'Mid' if value contains 'Mid'
mentor_df['mentor_circle_preference'] = mentor_df['mentor_circle_preference'].apply(assignCirclePreference)


In [ ]:
#count distinct values of mentor circle preference
mentor_df['mentor_circle_preference'].value_counts()

In [ ]:
# Add a dummy column to both data frames  
mentor_df['dummy'] = 1  
mentee_df['dummy'] = 1  
  
# Perform a cross join by merging on the dummy column  
cross_join_df = pd.merge(mentor_df, mentee_df, on='dummy')  
  
# Remove the dummy column  
cross_join_df = cross_join_df.drop(columns=['dummy'])  
  
# Calculate the match score for each mentor-mentee pair  
# You can implement your own scoring algorithm or use any other method here  
  


In [ ]:
# Add an NoMatch column and populate it with 1 if mentor grade is less than mentee grade, otherwise populate it with 0
cross_join_df['NoMatch'] = np.where(cross_join_df['mentor_grade'] <= cross_join_df['mentee_grade'], 1, 0)

In [ ]:
#update nomatch olumn to 1 if mentor tmezone and mentee timezone difference is greater than 2
cross_join_df['NoMatch'] = np.where(abs(cross_join_df['mentor_timezone_num'] - cross_join_df['mentee_timezone_num']) > 3, 1, cross_join_df['NoMatch'])

In [ ]:
#add column mentee circle level as 'Early' if mentee experience is less than or equal to 8 'Mid otherwise
cross_join_df['mentee_circle_preference'] = np.where(cross_join_df['mentee_experience'] <= 8, 'Early', 'Mid')

#create column matchCirclePreference as 4 if mentor circle preference is equal to mentee circle preference, 0.25 otherwise  - give higher weightage to matching circle preference
cross_join_df['matchCirclePreference'] = np.where(cross_join_df['mentor_circle_preference'] == cross_join_df['mentee_circle_preference'], 4, 0.25)
#update nomatch when circle preference is not the same
cross_join_df['NoMatch'] = np.where(cross_join_df['mentor_circle_preference'] != cross_join_df['mentee_circle_preference'], 1, cross_join_df['NoMatch'])

In [ ]:
#get count of rows by NoMatch column
cross_join_df['NoMatch'].value_counts()

In [ ]:
#get distnct count of mentee ids in cross_join_df where NoMatch is 0
cross_join_df[cross_join_df['NoMatch'] == 0]['mentee_id'].nunique()

In [ ]:
#get the list of mentee ids that are in mentee_df but not in (cross_join_df['matching_circles] == 1)
dropped_mentee_df = mentee_df[~mentee_df['mentee_id'].isin(cross_join_df[cross_join_df['NoMatch'] == 0]['mentee_id'])]['mentee_id']
if dropped_mentee_df.shape[0] > 0:
    print('Mentee ids that are dropped:', dropped_mentee_df)
else:
    print('All mentee ids are present in the matching dataframe')    



In [ ]:
#write the cross_join_df to a csv file fo review
#cross_join_df.to_csv('../data/output/cross_join_df.csv', index=False)

In [ ]:
#create a new coulm matched_interests by applying common_interests function
cross_join_df['matched_interests'] = cross_join_df.apply(lambda x: common_interests(x['mentor_split_interests'], x['mentee_split_interests']), axis=1)
#create a new column matched_interests_count as count of items in matched_interests column
cross_join_df['matched_interests_count'] = cross_join_df['matched_interests'].apply(lambda x: len(x))

In [ ]:
#create matched_goals column by applying score_similarity_between_mentor_mentee_goals function
cross_join_df['matched_goals'] = cross_join_df.apply(lambda x: score_similarity_between_mentor_mentee_goals(x['mentor_goal'], x['mentee_goal']), axis=1)
#create matchGrowth column by applying matchGrowth function
cross_join_df['matchGrowth'] = cross_join_df.apply(lambda x: matchGrowth(x['mentor_cll'], x['mentee_cll']), axis=1)
#create matchGrowth count column as count of items in matchGrowth function
cross_join_df['matchGrowth_count'] = cross_join_df['matchGrowth'].apply(lambda x: len(x))

In [ ]:
cross_join_df.dtypes

In [ ]:

#create x-grade-level-score such that it is (2/(cross_join_df['mentor_grade'] - cross_join_df['mentee_grade']+1)) and round it to 2 decimal places
cross_join_df['x-grade-level-score'] = (2/(cross_join_df['mentor_grade'] - cross_join_df['mentee_grade']+1)).round(4)


#create x-timezone-score such that it is (2/(abs(cross_join_df['mentor_timezone_num'] - cross_join_df['mentee_timezone_num'])+1))
cross_join_df['x-timezone-score'] = (2/(abs(cross_join_df['mentor_timezone_num'] - cross_join_df['mentee_timezone_num'])+1)).round(4)
#create a score column such that if NoMatch is 1, score is -1, otherwise score is ((2/difference in mentor and mentee grade ) +matched_interests_count + (matched_goals * 2) + matchCLLGrowth_count + (2/difference in mentor and mentee timezone) +matchedcirclepreference)
#cross_join_df['score'] = np.where(cross_join_df['NoMatch'] == 1, -1, cross_join_df['x-grade-level-score'] + cross_join_df['matched_interests_count'] *0.5 + (cross_join_df['matched_goals'] * 2) + cross_join_df['matchCLLGrowth_count'] *0.5 + cross_join_df['x-timezone-score'] + cross_join_df['matchCirclePreference'])

cross_join_df['score'] = np.where(cross_join_df['NoMatch'] == 1, -1, cross_join_df['x-grade-level-score'] + cross_join_df['matched_interests_count'] *0.5 + (cross_join_df['matched_goals'] * 2) + cross_join_df['matchCLLGrowth_count'] *0.5 + cross_join_df['x-timezone-score'] )

#round the score to 4 decimal places
cross_join_df['score'] = cross_join_df['score'].round(4)

In [ ]:
#write the output to a csv file OVERWRITING the existing file
#cross_join_df.to_csv('../data/output/mentor_mentee_match.csv', index=False)

In [ ]:
#check if score is infinite or not
count = np.isinf(cross_join_df['score']).values.sum() 
print("It contains " + str(count) + " infinite values") 
#get the maximum value of score column
max_score = cross_join_df['score'].max()
#print the maximum value of score column
print("The maximum score is " + str(max_score))

In [ ]:
#create a new circle column and set its value as 0
cross_join_df['circle'] = 9999
cross_join_df['circle'].value_counts()


In [ ]:
#sort the cross_join_df by score column in descending order and filter out the rows with score less than 0
cross_join_df = cross_join_df.sort_values(by='score', ascending=False)
cross_join_df_filtered = cross_join_df[cross_join_df['score'] > 0]
print(cross_join_df_filtered.shape)


In [ ]:
#make a deep copy of cross_join_df and store it as original_cross_join_df
original_cross_join_df = cross_join_df.copy(deep=True)

In [ ]:
#Assign mentoring circles
mentoring_circles = []  
assigned_mentees = set()  
assigned_mentors = set()  

num_mentors = mentor_df.shape[0]  
subset_df = cross_join_df_filtered


#function to assign mentee to a circle
def assign_mentee(circle, mentee_id):  
    if mentee_id not in assigned_mentees:
        circle['mentees'].append(mentee_id)  
        assigned_mentees.add(mentee_id)  
        cross_join_df.loc[(cross_join_df['mentor_id'] == mentor_id) & (cross_join_df['mentee_id'] == mentee_id), 'circle'] = circle['circle_num']  

  
# Iterate over the rows of the subset data frame  
for _, row in subset_df.iterrows():  
    mentor_id = row['mentor_id']  
    mentee_id = row['mentee_id']  
    # Check if the mentor has already been assigned the maximum number of mentees
    if mentor_id not in assigned_mentors:
            circle_num= len(mentoring_circles) + 1
            circle = {'circle_num':circle_num, 'mentor': mentor_id, 'mentees': []}
            mentoring_circles.append(circle)
            assigned_mentors.add(mentor_id)

            assign_mentee(circle, mentee_id)
    else:  
            # Iterate over the existing circles
            for circle in mentoring_circles:  
                # Check if the mentor is already in the circle  and the circle has not reached the maximum number of mentees
                if circle['mentor'] == mentor_id and len(circle['mentees']) < max_mentees_per_mentor:  
                    assign_mentee(circle, mentee_id)

                      


print(mentoring_circles)

In [ ]:
#group the mentees by circle, circle_preference and count the number of mentees in each circle where circle is not 9999
grouped = cross_join_df[cross_join_df['circle'] != 9999].groupby(['circle', 'mentor_circle_preference'])['mentee_id'].count().reset_index()
#display the grouped data
print(grouped)

In [ ]:
print(cross_join_df.columns)

In [ ]:
#Get distinct menteeid where circle is not 9999
assignedmenteeids = cross_join_df[cross_join_df['circle'] != 9999]['mentee_id'].unique()
print('Number of mentees:', len(assignedmenteeids))


In [ ]:
#get the list of mentee ids that are in mentee_df but not in assignedmenteeids
unassigned_mentee_df = mentee_df[~mentee_df['mentee_id'].isin(assignedmenteeids)]['mentee_id']
if unassigned_mentee_df.shape[0] > 0:
    print('Mentee ids that are dropped:', unassigned_mentee_df)
else:
    print('All mentee ids are present in the matching dataframe') 

In [ ]:
orderofcolumns =['circle', 'score','mentor_id', 'mentor_grade',
 'mentor_name', 'mentor_first_name', 'mentor_last_name',
       'mentor_email', 'mentor_job_title',
       'mentee_id', 'mentee_name', 'mentee_first_name', 'mentee_last_name',
       'mentee_email','mentee_job_title','mentee_grade','mentor_grade','x-grade-level-score',
        'mentor_timezone', 'mentee_timezone', 'x-timezone-score',
       'NoMatch', 
       'mentee_circle_preference','mentor_circle_preference','mentee_experience','mentee_orig_experience',
       'matchCirclePreference', 
         'matched_goals', 'mentor_goal', 'mentee_goal','matched_interests',
       'matched_interests_count', 'mentor_interests', 'mentee_interests',
        'mentor_cll','mentee_cll', 'matchCLLGrowth', 'matchCLLGrowth_count',
       'mentor_eligibility', 'mentor_wiadspast_mentorship','mentee_mgr_approval',
       'mentee_eligibility', 'mentee_past_mentorship', 'mentee_wiads_member', 'mentor_org',
       'mentor_dept', 'mentor_GLcode', 'mentor_business_unit',
       'mentor_wiads_member',
       'mentor_past_mentorship' , 'mentee_org',   'mentee_dept', 'mentee_GLcode', 'mentee_business_unit']

In [ ]:
# subset_columns = ['circle', 'score','mentor_id', 'mentor_name', 'mentee_id', 'mentee_name', 'mentor_grade', 'mentee_grade', 'x-grade-level-score', 'mentee_circle_preference','mentor_circle_preference','mentee_experience',
       # 'matchCirclePreference',  'matched_goals', 'mentor_goal', 'mentee_goal','matched_interests',
       # 'matched_interests_count', 'mentor_interests', 'mentee_interests',
       #  'mentor_cll','mentee_cll', 'matchCLLGrowth',  'mentor_timezone', 'mentee_timezone','x-timezone-score','mentee_timezone_num','mentor_timezone_num']

In [ ]:

# #subselect column and sort by circle ascending and score descending
# subselected_df = cross_join_df[subset_columns].sort_values(by=['circle', 'score'], ascending=[True, False])
# #write the output to a csv file
# subselected_df.to_csv('../data/output/mentoring_circles.csv', index=False)





In [ ]:
#subset the cross_join_df with the order of columns
cross_join_df = cross_join_df[orderofcolumns]

In [ ]:
#sort cross_join_df by circle column
cross_join_df = cross_join_df.sort_values(by=['circle', 'score'], ascending=[True, False])
#Write the cross_join_df to a xlsx file and with order of columns specified
#orderofcolumns = ['mentor_id', 'mentee_id', 'score', 'NoMatch', 'matched_interests', 'matched_interests_count', 'matched_goals', 'matchCLLGrowth', 'matchCLLGrowth_count', 'mentee_circle_preference', 'mentor_circle_preference', 'matchCirclePreference', 'circle']
cross_join_df.to_excel('../data/output/mentor_mentee_circles_no_x_circle.xlsx', columns=orderofcolumns, index=False)